# EV Data: Electric Nation - Greenflux

The Electric Nation EV GreenFlux data comes in one big file. This notebook assumes that the data in the original file (_GreenFlux3Minute.csv_) has been separated into two different files according to the charger type (3.6 or 7 kW) using the Jupyter notebook **_EV_GreenFlux_separate_data_by_charger.ipynb_**.

The Greenflux data provides the charging current, which is resampled and converted to power, assuming a voltage of 230V. For generating the charging profiles, the function `resample_timeseries()` resamples the current timeseries as specified with the parameter `time_resolution` and converts the current to power. The function is based on `pd.DataFrame.resample()`and in this notebook, `mean()` is used to aggregate the data over the resampling interval.

This notebook works for _GreenFlux3Minute.csv_. There is also a version of the data with 15 min time resolution (note that some of the fields have different names). However, this is simply an averaged version of the 3 min data.

In [ ]:
from csv import reader
from pathlib import Path
import pandas as pd
from datetime import datetime, time
import logging

# Set up logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [ ]:
# define parameters
time_resolution = "30min"
charger_type = 7 # (not 7.0!!) or 3.6 kW
start_date = "2018-01-01"
end_date = "2018-12-31"
save_file = False

dates = pd.date_range(start=start_date,end=end_date).to_pydatetime()
dates = [x.date() for x in dates]

## Load data

In [ ]:
# load file
filename = f"EV_GreenFlux_3min_data_{charger_type}kW.csv"
path_to_file = Path.cwd() /""/ filename
data = pd.read_csv(path_to_file, low_memory=False)
# separate columns for time and date
data["TimeStamp"] = data["TimeStamp"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f") ) 
data["Date"] = data["TimeStamp"].apply(lambda x: x.date()) 
data["Time"] = data["TimeStamp"].apply(lambda x: x.time()) 

## Function to resample timeseries

In [ ]:
def resample_timeseries(df, date, charger_id, voltage=230):
    """ resamples the current for one charger and day and converts it to power.
    
    The current is averaged over the specified time resolution before converted to power.
    
    Parameters
    ----------
    df_event: pd.DataFrame
        The dataframe containing the timeseries of the pv output of one system and day
    system_id: int
        System ID of the PV system.
    date: datetime.time 
        Date of the timeseries.
     voltage: int, optional
        Voltage of the network, used to convert current into power.
        
    Returns
    -------
    timeseries: pd.DataFrame
        Dataframe containing the resampled power in kW.
    """
    timeseries = pd.DataFrame(df[["TimeStamp", "MaxAmpsUsed"]].set_index("TimeStamp").resample(time_resolution).mean())
    timeseries = timeseries.assign(timestamps = timeseries.index.time)
    timeseries = timeseries.set_index("timestamps")
    # convert current into power in kW
    timeseries["MaxAmpsUsed"] = timeseries["MaxAmpsUsed"]*voltage/1000.0
    timeseries = timeseries.rename(columns={"MaxAmpsUsed": f"{charger_id}_{date}"})
    timeseries.fillna(0, inplace=True)
    return timeseries



## Initialise dataframe for resampled charging events

In [ ]:
# create timestamps with defined time resolution
timestamps = pd.date_range("00:00", "23:59", freq=time_resolution) # change freq to modify time resolution

charging_profiles = pd.DataFrame(timestamps, columns=["Timestamps"]) 
charging_profiles["Timestamps"] = charging_profiles["Timestamps"].apply(lambda x: x.time() )
charging_profiles.set_index("Timestamps", inplace=True)

## Re-sample charging current

Filter the data for the provided time period and re-sample each charging event individually.

In [ ]:
for date in dates:
    data_filtered_for_date = data[data["Date"] == date]
    charger_ids = pd.unique( data_filtered_for_date["ChargerID"] )
    for charger in charger_ids:
        df = data_filtered_for_date[data_filtered_for_date["ChargerID"] == charger]
        if df["MaxAmpsUsed"].sum() > 0:
            timeseries=resample_timeseries(df, date, charger)
            charging_profiles = charging_profiles.join(timeseries)
charging_profiles.fillna(0, inplace=True)

## Save to csv

In [ ]:
charging_profiles

In [ ]:
# remove empty charging profiles 
if save_file:
    charging_profiles = charging_profiles.loc[:, (charging_profiles != 0).any(axis=0)]
    charging_profiles.to_csv(f"EV-Greenflux--{charger_type}kW--start-{start_date}--end-{end_date}--{time_resolution}.csv", index=False)